# Pytorch 기반 GoogLeNet 구현


In [ ]:
import torch
import torch.nn as nn

### inception module class

In [ ]:
class InceptionBlock(nn.Module):
   def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
       super(InceptionBlock, self).__init__()

       # 1x1 convolution branch
       self.branch1 = nn.Sequential(
           nn.Conv2d(in_channels, ch1x1, kernel_size=1),
           nn.ReLU(inplace=True) # inplace 위의 Conv2d의 결과 값을 덮어쓰기
       )

       # 1x1 convolution -> 3x3 convolution branch
       self.branch2 = nn.Sequential(
           nn.Conv2d(in_channels, ch3x3red, kernel_size=1),
           nn.ReLU(inplace=True),
           nn.Conv2d(ch3x3red, ch3x3, kernel_size=3, padding=1),
           nn.ReLU(inplace=True)
       )

       # 1x1 convolution -> 5x5 convolution branch
       self.branch3 = nn.Sequential(
           nn.Conv2d(in_channels, ch5x5red, kernel_size=1),
           nn.ReLU(inplace=True),
           nn.Conv2d(ch5x5red, ch5x5, kernel_size=5, padding=2),
           nn.ReLU(inplace=True)
       )

       # 3x3 max pooling -> 1x1 convolution branch
       self.branch4 = nn.Sequential(
           nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
           nn.Conv2d(in_channels, pool_proj, kernel_size=1),
           nn.ReLU(inplace=True)
       )

   def forward(self, x):
       branch1 = self.branch1(x)
       branch2 = self.branch2(x)
       branch3 = self.branch3(x)
       branch4 = self.branch4(x)

       # Concatenate along the channel dimension
       outputs = [branch1, branch2, branch3, branch4]
       return torch.cat(outputs, 1) # 1 == c / b, c, h, w

### Auxilary classifier class

In [ ]:
class AuxiliaryClassifier(nn.Module):
   def __init__(self, in_channels, num_classes):
       super(AuxiliaryClassifier, self).__init__()

       self.avgpool = nn.AvgPool2d(kernel_size=5, stride=3)
       self.conv = nn.Conv2d(in_channels, 128, kernel_size=1)
       self.relu = nn.ReLU(inplace=True)
       self.fc1 = nn.Linear(2048, 1024)
       self.dropout = nn.Dropout(p=0.7)
       self.fc2 = nn.Linear(1024, num_classes)

   def forward(self, x):
       x = self.avgpool(x)
       x = self.conv(x)
       x = self.relu(x)
       x = torch.flatten(x, 1)
       x = self.fc1(x)
       x = self.relu(x)
       x = self.dropout(x)
       x = self.fc2(x)
       return x

### main modeling class

In [ ]:
def train():
   device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
   model = GoogLeNet(num_classes=1000, aux_logits=True).to(device)

   # Loss and optimizer
   criterion = nn.CrossEntropyLoss()
   optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

   # Training loop
   model.train()
   for epoch in range(num_epochs):
       for inputs, labels in train_loader:
           inputs = inputs.to(device)
           labels = labels.to(device)

           # Forward pass
           if model.aux_logits:
               outputs, aux1, aux2 = model(inputs)
               loss1 = criterion(outputs, labels)
               loss2 = criterion(aux1, labels)
               loss3 = criterion(aux2, labels)
               loss = loss1 + 0.3*(loss2 + loss3)
           else:
               outputs = model(inputs)
               loss = criterion(outputs, labels)

           # Backward and optimize
           optimizer.zero_grad()
           loss.backward()
           optimizer.step()

In [ ]:
class GoogLeNet(nn.Module):
   def __init__(self, num_classes=1000, aux_logits=True):
       super(GoogLeNet, self).__init__()

       self.aux_log its = aux_logits

       # Initial layers
       self.conv1 = nn.Sequential(
           nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
           nn.ReLU(inplace=True),
           nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
       )

       self.conv2 = nn.Sequential(
           nn.Conv2d(64, 64, kernel_size=1),
           nn.ReLU(inplace=True),
           nn.Conv2d(64, 192, kernel_size=3, padding=1),
           nn.ReLU(inplace=True),
           nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
       )

       # Inception blocks
       self.inception3a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
       self.inception3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
       self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

       self.inception4a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
       self.inception4b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
       self.inception4c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
       self.inception4d = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
       self.inception4e = InceptionBlock(528, 256, 160, 320, 32, 128, 128)
       self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

       self.inception5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
       self.inception5b = InceptionBlock(832, 384, 192, 384, 48, 128, 128)

       # Auxiliary classifiers
       if self.aux_logits:
           self.aux1 = AuxiliaryClassifier(512, num_classes)
           self.aux2 = AuxiliaryClassifier(528, num_classes)

       # Final layers
       self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
       self.dropout = nn.Dropout(0.4)
       self.fc = nn.Linear(1024, num_classes)

       # Weight initialization
       self._initialize_weights()

   def forward(self, x):
       # N x 3 x 224 x 224
       x = self.conv1(x)
       # N x 64 x 112 x 112
       x = self.conv2(x)
       # N x 192 x 56 x 56
       x = self.inception3a(x)
       # N x 256 x 56 x 56
       x = self.inception3b(x)
       # N x 480 x 56 x 56
       x = self.maxpool3(x)
       # N x 480 x 28 x 28
       x = self.inception4a(x)
       # N x 512 x 28 x 28

       if self.training and self.aux_logits:
           aux1 = self.aux1(x)

       x = self.inception4b(x)
       x = self.inception4c(x)
       x = self.inception4d(x)

       if self.training and self.aux_logits:
           aux2 = self.aux2(x)

       x = self.inception4e(x)
       x = self.maxpool4(x)
       x = self.inception5a(x)
       x = self.inception5b(x)

       x = self.avgpool(x)
       x = torch.flatten(x, 1)
       x = self.dropout(x)
       x = self.fc(x)

       if self.training and self.aux_logits:
           return x, aux1, aux2
       return x

   def _initialize_weights(self):
       for m in self.modules():
           if isinstance(m, nn.Conv2d):
               nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
               if m.bias is not None:
                   nn.init.constant_(m.bias, 0)
           elif isinstance(m, nn.Linear):
               nn.init.normal_(m.weight, 0, 0.01)
               nn.init.constant_(m.bias, 0)